# Day 19: Medicine for Rudolph

https://adventofcode.com/2015/day/19

## Part 1

In [261]:
def stringToElements(s):
    sv = []
    i = 0
    while i<len(s):
        if i<len(s)-1 and s[i+1].islower():
            sv.append(s[i:i+2])
            i+=2
        else:
            sv.append(s[i])
            i+=1
    return sv   

#stringToElements("HOH")
stringToElements("NRnFAr")

['N', 'Rn', 'F', 'Ar']

In [286]:
def getMolAndRepl(filename):
    f = open(filename)
    r,s = f.read().split("\n\n")
    s = s.strip('\n')
    repl = {}
    for l in r.split('\n'):
        t = l.strip("\n").split(" => ")
        if t[0] in repl.keys():
            repl[t[0]].append( t[1] )
        else:
            repl[t[0]] = [ t[1] ]
    return s,repl

s, repl = getMolAndRepl("./data/day19test0.txt")
#s, repl = getMolAndRepl("./data/input19.txt")
s, repl

('HOH', {'H': ['HO', 'OH'], 'O': ['HH']})

In [320]:
from collections import defaultdict

def replaceElements(s,repl):

    reps = defaultdict(lambda:0)
    sv = stringToElements(s)

    for i in range(len(sv)):
        rep = ""
        if i>0:
            for j in range(0,i):
                rep += sv[j]
        repi = []
        if sv[i] in repl.keys():
            for r in repl[sv[i]]:
                repi.append(rep+r)
        else:
            repi.append(rep+sv[i])
        if i<len(sv)+1:
            for j in range(i+1,len(sv)):
                for k in range(len(repi)):
                    repi[k] = repi[k]+sv[j]
        for r in repi:
            if r != s: 
                # don't count molecules that are copies of initial one
                # because of elements with no replacement
                reps[r] += 1
    
    return reps

s, repl = getMolAndRepl("./data/input19.txt")
reps = replaceElements(s,repl)
print(len(reps))

535


In [326]:
def replaceElementsReverse(s,repl):
    reps = defaultdict(lambda:0)
    sv = stringToElements(s)
    for i in range(len(sv)-1,-1,-1):        
        sbefore = sv[:i]
        safter  = sv[i+1:]
        repi = []
        if sv[i] in repl.keys():
            for r in repl[sv[i]]:
                repi.append(sbefore+stringToElements(r)+safter)
        else:
            repi.append(sv)
        for r in repi:
             if r != sv: 
                reps["".join(r)] += 1
    return reps

s, repl = getMolAndRepl("./data/input19.txt")
reps = replaceElementsReverse(s,repl)
print(len(reps))

535


## Part 2

Even with pruning of solutions, backtracking seems to take too much: tree is too large!

I need to find some pattern to simplify the problem and/or improve the pruning...

Target molecule finishes in `F`, many replacement rules produce molecules finishing in `Ar`, and there's no replacement rule for `Ar`, so any molecule finishing in `Ar` could never become the targe molecule. This could be generalized to molecule with no replament rules.

Maybe I should start the replacement procedure from the last element and check the tail element first?

In [330]:
from copy import deepcopy

#s, repl = getMolAndRepl("./data/day19test1.txt")
s, repl = getMolAndRepl("./data/input19.txt")

#s = "HOHOHO"

sv = stringToElements(s)
mol = {'e' : 1}
steps = 0

while s not in mol.keys():
    
    newmol = defaultdict(lambda:0)
    
    for m in mol.keys():

        for r in (replaceElementsReverse(m,repl)).keys():
            
            # skipping molecule step already saved
            if r in newmol.keys():
                continue
            
            # pruning of molecules that could not lead to final one
            addToNext = True
            sr = stringToElements(r)
            
            # last element 
            if sr[-1]!='F' and sr[-1] not in repl.keys():
                addToNext = False            
            else:
                for e1,e2 in zip(sr,sv[:len(sr)]):
                    # not equal to initial fraction of final one, and cannot change with replacement
                    if e1!=e2 and e1 not in repl.keys(): 
                        addToNext = False
                        break
            
            if addToNext: # and r not in testmol.keys():
                newmol[r] += 1
                
    mol = deepcopy(newmol)
    steps +=1
    print(steps, len(mol))

    # stop iterations: this would never converge in time...
    #if steps == 8:
    #    print(mol)
    #    break

print("Steps Part 2 =",steps)

1 3
2 6
3 14
4 31
5 70
6 157
7 353
8 793
9 1782
10 4004
11 8997
12 20216
13 45425


KeyboardInterrupt: 